In [ ]:
import HPreprocess as P
import HDataset as D
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from numpy import linalg as LA
from numpy.random import permutation
import cv2
import pickle
import random
ref = random.sample(range(10000, 40000),1)[0]

data = D.txt_to_var('test250.txt')
x_train = data[500:2200]
x_test = data[2500:3500]

def balance_data(dataRaw):
    label,train = zip(*dataRaw)
    labelf = [0 if i=='2' else 1 for i in label]
    nr = np.array(list(set(zip(labelf,train))))
    dataone = [data for data in nr if data[0]=='1']
    datazero = [data for data in nr if data[0]=='0']
    dataone = dataone[:len(datazero)]
    dataall = dataone + datazero
    permutation(dataall)
    return dataall

def feature_extract(img,nfea=300):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures = nfea,
    scaleFactor = 1.2,
    nlevels = 8,
    edgeThreshold = 15,
    firstLevel = 1,
    WTA_K = 2,
    patchSize = 30,
    fastThreshold = 10 
    )

    kp = orb.detect(img,None)
    kp, des = orb.compute(img, kp)
    des = np.array(des)
    if(des.shape[0] != nfea):
        des = np.concatenate((des,np.zeros(((nfea-des.shape[0],32)))),axis=0)
    scaler = StandardScaler()
    desf_s = scaler.fit_transform(des)
    return desf_s
    
xxxxxx    
x_train = balance_data(x_train)

xtrain, ytrain_label = D.single_fit(data=x_train,shape=(224,224),shuffleIm=True,num_class=2 )

xtrainCanny = xtrain.copy()

    
#xtrain_vec = [i.flatten() for i in xtrain]
orbfeature_train = list()
for i in range(len(xtrain)):
    orbfeature_train.append(feature_extract(img=xtrain[i],nfea=200))
    
orbfeature_train_vec = np.array([i.flatten() for i in orbfeature_train])


pca = PCA(n_components=1500)
pca_train_result = pca.fit_transform(orbfeature_train_vec)   


forest = RandomForestClassifier(n_estimators=500)
forest.fit(pca_train_result, ytrain_label)

neigh = KNeighborsClassifier(n_neighbors=50)
neigh.fit(pca_train_result, ytrain_label) 


In [ ]:
import pickle
datat = D.txt_to_var('test250.txt')
xtest, ytest_label = D.single_fit(data=datat,shape=(224,224),shuffleIm=True,num_class=4 )

yadap = ytest_label.argmax(-1)
yadap = [0 if i==2 else 1 for i in yadap]
yadap = np.array(yadap).astype('uint8')


neigho = open('savedmodel/neigh_model_34209.sav','rb')
foresto = open('savedmodel/forest_model_34209.sav','rb')
pca = PCA(n_components=1500)
neigh = pickle.load(neigho)
forest = pickle.load(foresto)

orbfeature_test = list()
for i in range(len(xtest)):
    orbfeature_test.append(feature_extract(img=xtest[i],nfea=200))
    
orbfeature_test_vec = np.array([i.flatten() for i in orbfeature_test])
pca_test_result = pca.fit_transform(orbfeature_test_vec)  

forest_test_softpredictions = forest.predict_proba(pca_test_result)
neigh_test_softpredictions = neigh.predict_proba(pca_test_result)


forest_test_predictions = forest.predict(pca_test_result)
neigh_test_predictions = neigh.predict(pca_test_result)


In [ ]:
np.array(orbfeature_test).shape

In [ ]:
# Create first network with Keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
import numpy

# create model
model = Sequential()
model.add(Dense(256, input_dim=798, activation='relu'))
model.add(Dropout(0.5, noise_shape=None, seed=None))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5, noise_shape=None, seed=None))
model.add(Dense(32,  activation='relu'))
model.add(Dropout(0.5, noise_shape=None, seed=None))
model.add(Dense(1,  activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(pca_train_result, ytrain_label[:,1], epochs=30, batch_size=32)
# calculate predictions
predictions = model.predict(pca_test_result, batch_size=32)


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(forest_test_predictions.argmax(-1),yadap)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def mulit_roc_plot(yTrue,yPred,numClass):
    n_classes =numClass
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(yTrue[:, i], yPred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    colors = ['blue', 'red', 'green','yellow','orange']
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, 
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for multi-class data')
    plt.legend(loc="lower right")
    plt.show()
    
mulit_roc_plot(yTrue=yadap,yPred=,numClass=1) 

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(yadap, forest_test_softpredictions[1][:,1].reshape(-1,1))
roc_auc = auc(fpr, tpr)
plt.plot(fpr,tpr,label='ROC curve of class {0} (area = {1:0.4f})'
                 ''.format(i, roc_auc))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()